In [ ]:
import scipy.io as sio
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
%matplotlib inline

from pprint import pprint
from keras import regularizers
from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.applications.xception import (
    Xception, preprocess_input, decode_predictions)

In [ ]:
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', -1)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')
    
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err
 
def compare_images(imageA, imageB, title):
    # compute the mean squared error and structural similarity
    # index for the images
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB)
 
    # setup the figure
    fig = plt.figure(title)
    plt.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))
 
    # show first image
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(imageA, cmap = plt.cm.gray)
    plt.axis("off")
 
    # show the second image
    ax = fig.add_subplot(1, 2, 2)
    plt.imshow(imageB, cmap = plt.cm.gray)
    plt.axis("off")
 
    # show the images
    plt.show()

In [ ]:
model = Xception(
    include_top=True,
    weights='imagenet')
image_size = (299, 299, 3)

In [ ]:
VID = pd.read_csv("vehicle_categories.txt", sep='\t')
CID = pd.read_csv("clothing_categories.txt", sep='\t')
VID.head()

In [ ]:
path = os.path.join("./Car Data/cam_images2")

def newest(path):
    files = os.listdir(path)
    #print(files)
    paths = [os.path.join(path, basename) for basename in files]
    recentPath=max(paths, key=os.path.getmtime)
    print(recentPath)
    return recentPath

print(newest(path))
print(len(os.listdir(path)))

In [ ]:
#Primary Model
data2 = []

for y in range(400):
    z = y+1
    image_path = os.path.join("./Car Data/", "cam_images2", "cam_img (%s).jpg" %(z))
    #image_path = newest(path)
    img = image.load_img(image_path, target_size=image_size)

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    predictions = model.predict(x)
    array = []
    prob = 0
    ratio = 1
    result = decode_predictions(predictions, top=100)
    topresult = result[0][0]
#    print(topresult)
    while ratio > 0.01:
        currentresult = result[0][len(array)]
        prob = currentresult[2]
        #collect probabilities
        array.append(prob)
        #calculate the ratio of probability vs cumulative probability
        ratio = prob/sum(array)
#        print(result, sum(array), ratio)
    print(z, len(array))
#    result1 = decode_predictions(predictions, top=1)[0][0]
    result2 = result[0][1]
    result3 = result[0][2]
#    print(z,topresult,result2,result3)
    G = gini(np.asarray(array))*100
    C1 = prob*100
    Weight = (np.sqrt((G * G) + (C1 * C1)))/np.sqrt(2)
    
    if (topresult[0] in VID['Serial'].values) or (topresult[0] in CID['Serial'].values):
        if (topresult[0] in VID['Serial'].values):
            imgclass = "Vehicle 1"# + str((round(topresult[2]*1000))/10)
            classp = str((round(topresult[2]*1000))/10)
        else:
            imgclass = "Person 1"# + str((round(topresult[2]*1000))/10)
            classp = str((round(topresult[2]*1000))/10)
    elif (result2[0] in VID['Serial'].values) or (result2[0] in CID['Serial'].values):
        if (result2[0] in VID['Serial'].values):
            imgclass = "Vehicle 2"# + str((round(result2[2]*1000))/10)
            classp = str((round(result2[2]*1000))/10)
        else:
            imgclass = "Person 2"# + str((round(result2[2]*1000))/10)
            classp = str((round(result2[2]*1000))/10)
    elif (result3[0] in VID['Serial'].values) or (result3[0] in CID['Serial'].values):
        if (result3[0] in VID['Serial'].values):
            imgclass = "Vehicle 3"# + str((round(result3[2]*1000))/10)
            classp = str((round(result3[2]*1000))/10)
        else:
            imgclass = "Person 3"# + str((round(result3[2]*1000))/10)
            classp = str((round(result3[2]*1000))/10)
    else:
        imgclass = "Other"# + str((round(topresult[2]*1000))/10)
        classp = str((round(topresult[2]*1000))/10)
    
    #    data2.append([z,G,result1[0],result1[1],(round(result1[2]*1000))/10,result2[0],result2[1],(round(result2[2]*1000))/10])
    data2.append([G,Weight,topresult[0],topresult[1],(round(topresult[2]*1000))/10,imgclass, classp])

#plt.imshow(img)
#data2 = pd.DataFrame(data2, columns = ['Gini','Weight','Serial 1','P 1','C 1,','Serial 2','P 2','C 2'])                 
data2 = pd.DataFrame(data2, columns = ['Gini','Weight','S 0','P 0','C 0','Class','Class P'])
#data2.head()

In [ ]:
print(data2['Weight'].mean()) #.loc[data2['Prediction 1']=="jeep",:]
print_full(data2)

In [ ]:
data2['Class'].value_counts()

In [ ]:
from skimage import io, measure
from skimage.measure import compare_ssim

import cv2

#original 720x1280, cropped 424x753
image_size = (424,753)


imgpath1 = io.imread('./Car Data/cam_images2/cam_img (23).jpg')
imgpath2 = io.imread('./Car Data/cam_images2/cam_img (118).jpg')
plt.imshow(imgpath1)

In [ ]:
print(mse(imgpath1,imgpath2))
plt.imshow(imgpath2)
compare_ssim(imgpath1,imgpath2,multichannel=True)

In [ ]:
comparison = []

for y in range(573):
    z = y+1
    image_path = os.path.join("./Car Data/", "cam_images", "cam_img (%s).jpg" %(z))

    imgpath1 = io.imread('./Car Data/cam_images/cam_img (1).jpg')
    imgpath2 = io.imread('./Car Data/cam_images/cam_img (%s).jpg' %(z))

    comparison.append([z,mse(imgpath1,imgpath2),compare_ssim(imgpath1,imgpath2,multichannel=True)])
comparison2 = pd.DataFrame(comparison, columns = ['Photo','MSE','SSIM'])

In [ ]:
print(comparison2)

In [ ]:
#Load scenario images
imgS1 = io.imread('./Car Data/cam_base/S1.jpg')
imgS2 = io.imread('./Car Data/cam_base/S2.jpg')
imgS3 = io.imread('./Car Data/cam_base/S3.jpg')
imgS4 = io.imread('./Car Data/cam_base/S4.jpg')
imgS5 = io.imread('./Car Data/cam_base/S5.jpg')
imgS6 = io.imread('./Car Data/cam_base/S6.jpg')
imgS7 = io.imread('./Car Data/cam_base/S7.jpg')
imgS8 = io.imread('./Car Data/cam_base/S8.jpg')
imgS9 = io.imread('./Car Data/cam_base/S9.jpg')
imgS10 = io.imread('./Car Data/cam_base/S10.jpg')
imgS11 = io.imread('./Car Data/cam_base/S11.jpg')
imgS12 = io.imread('./Car Data/cam_base/S12.jpg')
imgS13 = io.imread('./Car Data/cam_base/S13.jpg')
imgS14 = io.imread('./Car Data/cam_base/S14.jpg')
imgS15 = io.imread('./Car Data/cam_base/S15.jpg')
imgS16 = io.imread('./Car Data/cam_base/S16.jpg')
imgS17 = io.imread('./Car Data/cam_base/S17.jpg')
imgS18 = io.imread('./Car Data/cam_base/S18.jpg')

Sarray = [0,imgS1,imgS2,imgS3,imgS4,imgS5,imgS6,imgS7,imgS8,imgS9,imgS10,imgS11,imgS12,imgS13,imgS14,imgS15,imgS16,imgS17,imgS18]

print(Sarray)

In [ ]:
from skimage import io, measure
from skimage.measure import compare_ssim

import cv2

#original 720x1280, cropped 424x753
image_size = (424,753)

pic = 37
resultarray = []
for x in range(18):
    imgpath1 = io.imread('./Car Data/cam_images2/cam_img (%s).jpg' %(pic))
    imgpath2 = io.imread('./Car Data/cam_base/S%s.jpg' %(x+1))
    combomse = round(mse(imgpath1,imgpath2))
    combossim = round(compare_ssim(imgpath1,imgpath2,multichannel=True)*1000)/10
    resultarray.append([x+1,combomse,combossim])
resultdf = pd.DataFrame(resultarray, columns = ['Scenario','MSE','SSIM']) 

#find best match
Y = str(resultdf.loc[resultdf['SSIM'].idxmax()][0]).split('.')[0]
print(Y)
print(resultdf.loc[resultdf['SSIM'].idxmax()][1])


fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(imgpath1)
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(imgpath2)
resultdf.head(18)


In [ ]:
#run all photos through 
#This process takes roughly half an hour for each 500 photos.
comparray = []
#range is 5814
i1 =0

#cycle through photo backlog
for img1 in range(573):
    i1+=1
    i2 =0
    print(i1)
    
    #cycle through all scenarios for each photo
    resultarray = []
    for S in range(18):
        i2+=1
        imgpath = io.imread('./Car Data/cam_images2/cam_img (%s).jpg' %(img1+1))
        scenario = Sarray[i2]
        combomse = round(mse(imgpath,scenario))
        combossim = round(compare_ssim(imgpath,scenario,multichannel=True)*1000)/10
    
        resultarray.append([i2,combomse,combossim])
    resultdf = pd.DataFrame(resultarray, columns = ['Scenario','MSE','SSIM']) 

    #find best match
    Y = int(str(resultdf.loc[resultdf['SSIM'].idxmax()][0]).split('.')[0])
    Bestmatch = Y
    Bestmse = resultdf.loc[resultdf['SSIM'].idxmax()][1]
    Bestssim = resultdf.loc[resultdf['SSIM'].idxmax()][2]
    #Mark high & Low
    if Bestssim >90:
        initialflag = 1
    elif Bestssim < 30:
        initialflag = 2
    else:
        initialflag = 0
    comparray.append([i1,Bestmatch,Bestmse,Bestssim,initialflag])

compdf = pd.DataFrame(comparray, columns = ['Image', 'Best Match','MSE','SSIM','Flag'])        
print("done!") 

In [ ]:
#compdf.loc[compdf['Img 1'],:].values
#compdf['SSIM']
gini(compdf.as_matrix(columns=compdf.columns[3:4]))
print(compdf)

In [ ]:
comp2df = compdf
print(comp2df)

In [ ]:
#export_csv = compdf.to_csv ('./export_dataframe.csv', index = None, header=True)
comp2df['Flag'].value_counts()
comp2df.loc[comp2df['Flag']==1,['SSIM']].mean()
comp2df.loc[comp2df['Flag']==1,['SSIM']].count()

matches = []
mismatch = [0,0,0]

print(matches)

Average = comp2df.loc[comp2df['Flag']==1,['SSIM']].count() * comp2df.loc[compdf['Flag']==1,['SSIM']].mean()

matches.append(round(Average[0]*10)/10)

Testcount = comp2df.loc[comp2df['Flag']==1,['SSIM']].count()
matches.append(Testcount[0])

matches.append(round(((matches[0]/matches[1])*10)/10))

print(matches, mismatch)

In [ ]:
#Go through base image comparisons and manually validate.  It will update average SSIM match % for matches and mismatches 
#as you go.


#for x in compdf.index: #run through entire photo list
for x in range(40): #run through subset
#    if comp2df['Flag'][x] == 0: #disregard pre-flagged matches/mismatches
    print(x, comp2df['SSIM'][x], comp2df['Flag'][x])
    imgpath1 = io.imread('./Car Data/cam_images2/cam_img (%s).jpg' %(compdf['Image'][x]))
    imgpath2 = Sarray[compdf['Best Match'][x]]
    fig = plt.figure()
    print("Image ",x,"Scenario ",compdf['Best Match'][x])
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(imgpath1)
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(imgpath2)
    plt.show()
    Y = input("1=Y, 2=N : ")
    comp2df['Flag'][x] = Y
    print(comp2df['Flag'][x])
    
    #update match & mismatch counts & averages
    if comp2df['Flag'][x] == 1:
        matches[0]+=comp2df['SSIM'][x]
        matches[1] = comp2df.loc[comp2df['Flag']==1,:].count()
        matches[2] = round(((matches[0]/matches[1])*10)/10)
    elif comp2df['Flag'][x] == 2:
        mismatch[0]+=comp2df['SSIM'][x]
        mismatch[1] = comp2df.loc[comp2df['Flag']==2,:].count()
        mismatch[2] = round(((mismatch[0]/mismatch[1])*10)/10)
    print(matches,mismatch)

In [ ]:
matches

In [ ]:
mismatch